In [14]:
color_1 = '#C905C9'
color_2 = '#170C17'
color_3 = '#94C9A2'
shadow_size = 2
split = 90
a_mask = 165
input_path = "profilea.png"

In [15]:
from PIL import Image
with Image.open(input_path) as img:
  shadow_size = int(img.width*0.04)

In [16]:
!rm -r hover
!rm -r meta
!rm -r shake
!mkdir -p meta && unzip -d meta meta.zip
!mkdir hover
!mkdir shake


Archive:  meta.zip
  inflating: meta/filters/1.png      
  inflating: meta/filters/2.png      
  inflating: meta/filters/3.png      
  inflating: meta/filters/4.png      
  inflating: meta/shapes/1.png       
  inflating: meta/shapes/2.png       
  inflating: meta/shapes/3.png       
  inflating: meta/shapes/4.png       
  inflating: meta/shapes/5.png       
  inflating: meta/shapes/6.png       


In [17]:
import os
from PIL import Image

def apply_filters(image_path, split):
    # Set the colors using environment variables
    color1 = os.environ.get('color1', color_1)
    color2 = os.environ.get('color2', color_2)

    # Open the image
    with Image.open(image_path) as img:
        # Split the pixels into two groups based on brightness
        img = img.convert("RGBA")
        pixels = img.load()
        width, height = img.size
        for i in range(width):
            for j in range(height):
                r, g, b, a = pixels[i, j]
                brightness = (0.299 * r + 0.587 * g + 0.114 * b)
                if a != 0:
                    if brightness > split:
                        # Apply the pink filter to bright pixels
                        r_mask, g_mask, b_mask = tuple(int(color_1[i:i+2], 16) for i in (1, 3, 5))
                        r, g, b, a = pixels[i, j]
                        r_new = (r * (255 - a_mask) + r_mask * a_mask) // 255 # blend red channel with mask
                        g_new = (g * (255 - a_mask) + g_mask * a_mask) // 255 # blend green channel with mask
                        b_new = (b * (255 - a_mask) + b_mask * a_mask) // 255 # blend blue channel with mask
                        pixels[i, j] = (r_new, g_new, b_new, 255) # set the new color with full opacity
                    else:
                        # Apply the black filter to dark pixels
                        pixels[i, j] = (int(color2[1:3], 16) - int(r*0.2), int(color2[3:5], 16) - int(g*0.2), int(color2[5:7], 16) - int(b*0.3), 255)
        # Add the green shadow to the left of the image
        shadow_width = 50
        shadow = Image.new("RGBA", (shadow_width, height), (148, 201, 162, 128))
        img.paste(shadow, box=(-shadow_width, 0))
        # Save the result
        return img


In [18]:
from PIL import Image, ImageDraw

def add_left_shadow(im, shadow_size):
    # Open the image and get its dimensions
      width, height = im.size

      # Create a new image with the shadow size added to the width
      new_width = width + shadow_size
      new_im = Image.new('RGBA', (new_width, height), (0, 0, 0, 0))

      # Paste the original image onto the new image, shifted to the right by the shadow size
      new_im.paste(im, (shadow_size, 0))

      # Draw the shadow onto the new image
      draw = ImageDraw.Draw(new_im)
      shadow_color = (148, 201, 162, 255)  # #94C9A2 in RGBA format
      for y in range(height):
          left_pixel = None
          for x in range(shadow_size, new_width):
              pixel = new_im.getpixel((x, y))
              if pixel[3] > 0:
                  # We've found the leftmost non-transparent pixel in this row
                  left_pixel = x
                  break
          if left_pixel is not None:
              for x in range(shadow_size):
                  draw.point((left_pixel - x - 1, y), shadow_color)

      # Return the new image with the shadow added
      return new_im


In [19]:
from PIL import Image, ImageDraw

def add_right_shadow(im, shadow_size):
    # Open the image and get its dimensions
      width, height = im.size

      # Create a new image with the shadow size added to the width
      new_width = width + shadow_size
      new_im = Image.new('RGBA', (new_width, height), (0, 0, 0, 0))

      # Paste the original image onto the new image
      new_im.paste(im, (0, 0))

      # Draw the shadow onto the new image
      draw = ImageDraw.Draw(new_im)
      shadow_color = tuple(int(color_3[i:i+2], 16) for i in (1, 3, 5)) + (255,)
      for y in range(height):
          right_pixel = None
          for x in range(width - 1, -1, -1):
              pixel = new_im.getpixel((x, y))
              if pixel[3] > 0:
                  # We've found the rightmost non-transparent pixel in this row
                  right_pixel = x
                  break
          if right_pixel is not None:
              for x in range(shadow_size):
                  draw.point((right_pixel + x + 1, y), shadow_color)

      # Return the new image with the shadow added
      return new_im


In [20]:
from PIL import Image

def overlay_image_center(input_image):

    # Create a temporary image that is 1.25 times the size of the input image and has an alpha channel
    temp_image = Image.new('RGBA', (int(input_image.width * 1.25), int(input_image.height * 1.25)), (0, 0, 0, 0))

    # Compute the position to center the input image in the temporary image
    x = int((temp_image.width - input_image.width) / 2)
    y = int((temp_image.height - input_image.height) / 2)

    # Paste the input image onto the temporary image at the center position
    temp_image.paste(input_image, (x, y))

    # Return the temporary image with the input image centered as an overlay
    return temp_image


In [21]:
from PIL import Image

def apply_filter(original_image, filter_path: str) -> None:
    # Load the original image
    original_image = overlay_image_center(original_image)

    # Load the filter image and crop it to the size of the scaled up image
    filter_image = Image.open(filter_path)
    filter_width, filter_height = filter_image.size
    width, height = original_image.size
    if filter_width < width or filter_height < height:
        raise ValueError("Filter image is too small")
    crop_left = (filter_width - width) // 2
    crop_top = (filter_height - height) // 2
    crop_right = crop_left + width
    crop_bottom = crop_top + height
    cropped_filter = filter_image.crop((crop_left, crop_top, crop_right, crop_bottom))

    # Apply the filter to the scaled up image
    for x in range(width):
        for y in range(height):
            filter_pixel = cropped_filter.getpixel((x, y))
            inverted_pixel = tuple(255 - value for value in filter_pixel)
            if sum(filter_pixel) / 3 > 20:
              original_pixel = original_image.getpixel((x, y))
              if original_pixel[3] == 0:
                original_image.putpixel((x, y), inverted_pixel)
              else:
                # assume original_pixel and filter_pixel are tuples with (R,G,B) values
                new_pixel = (
                        original_pixel[0] - int(filter_pixel[0])*4,  # Average red values
                        original_pixel[1] - int(filter_pixel[1])*4,  # Average green values
                        original_pixel[2]- int(filter_pixel[2])*4,
                        original_pixel[3]  # Average blue values
                    )

                original_image.putpixel((x, y), new_pixel)

    # Save the resulting image
    return original_image

In [22]:
from PIL import Image

def crop_image(final_image, shape_path):
    # Open final and shape images
    shape_image = Image.open(shape_path)

    # Resize shape image to match final image size
    shape_image = shape_image.resize(final_image.size)

    # Create output image
    output_image = Image.new('RGBA', final_image.size)

    # Iterate through every pixel in final image
    for x in range(final_image.width):
        for y in range(final_image.height):
            # Check if corresponding pixel in shape image is not transparent
            if shape_image.getpixel((x, y)) != 0:
                # Copy pixel from final image to output image
                output_image.putpixel((x, y), final_image.getpixel((x, y)))

    # Save output image
    return output_image




In [23]:
def apply(image_path, filter_path, shape_path, shadow_size):
  if shadow_size >= 0:
    return crop_image(apply_filter(add_right_shadow(apply_filters(image_path, split), shadow_size), filter_path), shape_path)
  else:
    return crop_image(apply_filter(add_left_shadow(apply_filters(image_path, split), shadow_size * -1), filter_path), shape_path)


In [24]:
from PIL import Image

def overlay_image(input_image, output_image_path, state):
    # Open input image and get its dimensions
    input_width, input_height = input_image.size

    # Create transparent image that is 10% larger than input image
    transparent_width = int(input_width * 1.04)
    transparent_height = int(input_height * 1.04)
    transparent_image = Image.new('RGBA', (transparent_width, transparent_height), (0, 0, 0, 0))

    # Determine position to overlay input image based on state
    if state == 0:
        x = int((transparent_width - input_width) / 2)
        y = int((transparent_height - input_height) / 2)
    elif state == 1:
        x = transparent_width - input_width
        y = 0
    elif state == 2:
        x = int((transparent_width - input_width) / 2)
        y = 0
    elif state == 3:
        x = 0
        y = 0
    elif state == 4:
        x = 0
        y = int((transparent_height - input_height) / 2)
    elif state == 5:
        x = 0
        y = transparent_height - input_height
    elif state == 6:
        x = int((transparent_width - input_width) / 2)
        y = transparent_height - input_height
    elif state == 7:
        x = transparent_width - input_width
        y = transparent_height - input_height
    elif state == 8:
        x = transparent_width - input_width
        y = int((transparent_height - input_height) / 2)

    # Overlay input image on transparent image
    transparent_image.paste(input_image, (x, y))

    # Save output image
    transparent_image.save(output_image_path)


In [25]:
import zipfile
import os

# Path to the zip file
zip_path = 'meta.zip'
# Directory where files will be extracted
extract_dir = 'meta'

# Create directory if it doesn't exist
if not os.path.exists(extract_dir):
    os.makedirs(extract_dir)

# Open the zip file
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    # Extract all the contents into the directory
    zip_ref.extractall(extract_dir)



In [26]:
filter, shape = 1, 1
max_filter, max_shape = 4, 6
def place(output_path, state, shadow):
  global filter, shape
  overlay_image(apply(input_path, "meta/filters/"+str(filter)+".png", "meta/shapes/"+str(shape)+".png", int(shadow)),output_path, state)
  shape = shape + 1 if shape < max_shape else 1
  filter = filter + 1 if filter < max_filter else 1

In [27]:
#code for shake
place("shake/1.png", 8, 0)
place("shake/2.png", 0, shadow_size)
place("shake/4.png", 4, shadow_size/2)
place("shake/5.png", 0, -shadow_size)

In [28]:
#code for hover
place("hover/1.png", 1, 0)
place("hover/3.png",3, shadow_size/2)
place("hover/5.png",5, 0)
place("hover/6.png",6, -shadow_size/3)
place("hover/8.png",8, -shadow_size)

In [29]:
from PIL import Image

def gen_frame(path):
    im = Image.open(path)
    alpha = im.getchannel('A')

    # Convert the image into P mode but only use 255 colors in the palette out of 256
    im = im.convert('RGB').convert('P', palette=Image.ADAPTIVE, colors=255)

    # Set all pixel values below 128 to 255 , and the rest to 0
    mask = Image.eval(alpha, lambda a: 255 if a <=128 else 0)

    # Paste the color of index 255 and use alpha as a mask
    im.paste(255, mask)

    # The transparency index is 255
    im.info['transparency'] = 255

    return im

def create_gif(folder_path, output_path, duration):
    frames = []
    for file_name in sorted(os.listdir(folder_path)):
        if file_name.endswith('.png'):
            frames.append(gen_frame(os.path.join(folder_path, file_name)))
    frames[0].save(output_path, save_all=True, append_images=frames[1:], loop=0, duration=duration, optimize=True, disposal=2)

In [30]:
create_gif("shake", "shake.gif", 650)
create_gif("hover", "hover.gif", 550)